In [1]:
#IMPORTACIÓN DE LIBRERÍAS

import pandas as pd
import time
from string import ascii_uppercase as alfabeto
import pickle


#Para el WebScraping
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#Para las predicciones en el modelo
from scipy.stats import poisson

## **DATA COLLECTION**

En esta parte del notebook nos encargamos de acceder a los datos con la clase DataGetter, de allí obtenemos los datos en la tabla_ca. Luego lo exportamos a un archivo de pickle. Este archivo, llamado tabla_ca contiene las cuatro tablas de posiciones de la fase de grupos de la Copa América 2024. Posteriormente, con la clase DataCollector obtenemos los datos históricos de los partidos de la Copa América, por un lado usando WebScraping con bs4 y BeautifulSoup, por el otro accediendo al driver de Chrome usando selenium y realizando nuevamente WebScraping. Luego, exportamos los datos en tres archivos .csv que servirán posteriormente.

In [2]:
class DataGetter:
    '''
    Clase encargada de obtener los partidos de la Copa America 2024 y el Ranking Mundial de la FIFA actual.
    
    Atributos:
        url: path de la página Web.

    Métodos:
        charger: Se encarga de leer el path y entregar una lista tratable con pandas.
        organicer: Se encargada de organizar, filtrar y dejar listos los datos cargados. 
    '''
    def __init__(self, url):
        self.link = url #Link de la página web a la que accedemos
    
    def charger(self):
        '''
        Lee la página web y devuelve los datos en forma de lista.

        Retorna:
            data (list): Lista de DataFrames leídos de la página web.
        '''
        
        data = pd.read_html(self.link) #Leemos la página web
        
        return data

    def organicer(self, data):
        '''
        Organiza los datos en tablas por grupos.

        Parámetros:
            data (list): Lista de DataFrames a organizar.

        Retorna:
            dict_tables (dict): Diccionario con los datos organizados por grupos.
        '''
        
        dict_tables = {} #Creamos un diccionario vacío para rellenarlo con las tablas
        for letra, i in zip(alfabeto, range(14, 42, 7)): #Según el patrón encontrado, iteramos
            df = data[i] #Definimos el DataFrame para cada elemento encontrado en la web
            df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
            df.pop('Qualification') #Borramos columnas innecesarias
            dict_tables[f'Group {letra}'] = df #Definimos el nombre de cada grupo (A, B, C, D)
            
        for group in dict_tables:
            dict_tables[group]['Pts'] = 0

            #Renombramos algunas selecciones que presentaron problemas
            dict_tables['Group C'].loc[2, 'Team'] = 'United States'
            dict_tables['Group D'].loc[0, 'Team'] = 'Colombia'
            dict_tables['Group D'].loc[3, 'Team'] = 'Paraguay'
            
        return dict_tables

In [3]:
#Links con la info de la Copa América 2024 y el Ranking Mundial de la FIFA
url_ca = 'https://en.wikipedia.org/wiki/2024_Copa_America'

#Creamos los objetos que nos permiten extraer la data de los links correspondientes
Copa_America = DataGetter(url_ca)

#Leemos los datos de la web
datos_ca = Copa_America.charger()

#Organizamos y presentamos los datos previamente cargados
tabla_ca = Copa_America.organicer(datos_ca)

In [9]:
#Exportamos los datos de las tablas de la Copa América 2024
with open('tabla_ca', 'wb') as output:
    pickle.dump(tabla_ca, output)

In [10]:
class DataCollector:

    '''
    Clase diseñada para recolectar los datos de los partidos de cada edición de la Copa America 
    (datos históricos)

    Métodos: 
        
        - get_matches: Se encarga de utilizar WebScraping con bs4 para obtener los datos de los partidos
        de la Copa América en una edición específica.

        - getTotalMatches: Se encarga de recolectar todos los datos históricos de los partidos de la
        Copa América (todas las ediciones).

        -getMissingMatches: Se encarga de utilizar WebScraping con Selenium para obtener los datos faltantes,
        que no pudieron ser recolectados con bs4.

        -getTotalMissingMatches: Se encarga de recolectar los datos faltantes de todas las ediciones que no 
        pudieron ser obtenidas con bs4 (en este caso, eidiciones 2011 y 2015).

        
    '''
 
    def getMatches(self, year):
        
        '''
        Extrae los partidos de una página web usando técnicas de WebScraping.
        
        Parámetros:
            year (int): Año de la edición de Copa América.

        Retorna:
            df_America (pd.DataFrame): Data con los partidos extraídos de la página web.
        '''

        
        if year <= 1967:
            urls = f'https://en.wikipedia.org/wiki/{year}_South_American_Championship'
            
        else:
            urls = f'https://en.wikipedia.org/wiki/{year}_Copa_America'
            
        response = requests.get(urls) #Realizamos la solicitud de acceso a la web
        content = response.text #Extraemos el contenido en forma de texto
        
        soup = BeautifulSoup(content, 'lxml') #Damos acceso al scraping
        matches = soup.find_all('div', class_="footballbox") #Filtramos los datos que buscamos

        #Creamos tres listas vacías, para construir un diccionario
        home = []
        score = []
        away = []
        
        #Un ciclo for para cada elemento que cumpla la condición de filtrado y sea añadido a la lista matches
        for match in matches:
            #Llenamos las tres listas de acuerdo a la clase adecuada en el html de la web
            home.append(match.find('th', class_="fhome").get_text()) #Llenamos el home
            score.append(match.find('th', class_="fscore").get_text()) #Llenamos el score
            away.append(match.find('th', class_="faway").get_text()) #Llenamos el away
            
        dict_America = {'home':home, 
                        'score':score,
                        'away':away}

        #Creamos un DataFrame con los datos extraídos y con una columna adicional del año
        df_America = pd.DataFrame(dict_America)
        df_America['year'] = year

        return df_America
    
    def getTotalMatches(self, years):
        
        #Llenamos una lista con el total de partidos en todas las ediciones de la Copa América
        TotalMatches = [self.getMatches(year) for year in years]
        df_TotalMatches = pd.concat(TotalMatches, ignore_index=True) #Creamos el DF concatenando todos los partidos

        #DataFrame que almacena el histórico de partidos de la Copa América
        df_conmebol = df_TotalMatches[df_TotalMatches['year'] != 2024]
        #DataFrame que almacena el total de partidos de la edición 2024 (el fixture del torneo)
        df_fixture = df_TotalMatches[df_TotalMatches['year'] == 2024]

        return df_conmebol, df_fixture
    
    def getMissingMatches(self, year):
        service = Service(ChromeDriverManager().install()) #Instalamos (inicializamos) el Driver de Chrome
        driver = webdriver.Chrome(service=service) #Creamos el controlador (driver)
        
        url = f'https://en.wikipedia.org/wiki/{year}_Copa_America' #Link de la web
        
        driver.get(url) #Habilitamos el acceso al driver 
        
        #Indicamos al driver qué elementos extraer de la página web 
        missing_matches = driver.find_elements(by='xpath', value='//tr[@style="font-size:90%"]')

        #A partir de aquí, realizamos un procedimiento similar al de getMatches()
        home = []
        score = []
        away = []

        for match in missing_matches:
            home.append(match.find_element(by='xpath', value='./td[1]').text)
            score.append(match.find_element(by='xpath', value='./td[2]').text)
            away.append(match.find_element(by='xpath', value='./td[3]').text)

        dict_missing = {'home':home, 
                    'score':score,
                    'away':away}

        df_missing = pd.DataFrame(dict_missing)
        df_missing['year'] = year
        time.sleep(1) #Tiempo de espera para pasar de una página a la otra

        return df_missing
    
    def getTotalMissingMatches(self, years):
        #Todo similar a getTotalMatches
        missing_data = [self.getMissingMatches(year) for year in years]
        df_missing_data = pd.concat(missing_data, ignore_index=True)

        return df_missing_data

In [11]:
#Años de cada edición realizada por la Copa América
years_America = [1916, 1917, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1929, 1935, 1937, 1939, 1941,
                 1942, 1945, 1946, 1947, 1949, 1953, 1955, 1956, 1957, 1963, 1967, 1975, 1979, 1983, 1987, 1989,
                 1991, 1993, 1995, 1997, 1999, 2001, 2004, 2007, 2011, 2015, 2016, 2019, 2021, 2024]

missing_years = [2011, 2015]


Partidos = DataCollector() #Creamos el objeto que nos permitirá acceder a los datos históricos de los partidos
tablas = Partidos.getTotalMatches(years_America) #Datos con total de partidos obtenidos con bs4
tablas_perdidas = Partidos.getTotalMissingMatches(missing_years) #Datos con total de partidos obtenidos con Selenium

In [12]:
#Exportamos a archivos .csv los datos recolectados anteriormente
tablas[0].to_csv('Conmebol_Copa_America_initial_data.csv', index=False)
tablas[1].to_csv('Programacion_Copa_America_2024.csv', index=False)
tablas_perdidas.to_csv('Conmebol_Copa_America_missing_data.csv', index=False)

## **DATA CLEAN**

En esta parte del notebook nos encargamos de limpiar los datos. Con la clase DataCleanner, leemos los datos de los partidos totales históricos y del fixture de la Copa América 2024, además calculamos la fuerza de cada equipo de la copa.

In [13]:
class DataCleaner:
    '''
    Clase encargada de limpiar los datos obtenidos usando WebScraping.
    
    Atributos:
        - archivei con i = 1, 2, 3: Son los path de los archivos .csv con los datos necesarios
        
    Métodos:
        - readData: Se encarga de convertir los archivos .csv en un DataFrame de pandas.

        - cleanData: Se encarga de limpiar y transformar los datos para el uso adecuado.
    '''
    def __init__(self, archive1, archive2, archive3):
        
        #Cargamos los tres archivos a limpiar
        self.path1 = archive1
        self.path2 = archive2
        self.path3 = archive3
    
    def readData(self):
        
        #Leemos los tres archivos
        data1 = pd.read_csv(self.path1)
        data2 = pd.read_csv(self.path2)
        data3 = pd.read_csv(self.path3)
        
        return data1, data2, data3
    
    def cleanData(self, data, mode = 'fixture'):
        if mode == 'fixture': #Limpieza del Fixture de la Copa actual
            #Corregimos los espacios en blanco
            data[0]['home'] = data[0].home.str.strip()
            data[0]['away'] = data[0].away.str.strip()

            #Debido a que la página de Wikipedia ha sido actualizada, debemos editar el fixture
            for i in range(32):
                data[0].loc[i, 'score'] = f'Match {i+1}'

                data[0].loc[24, 'home'] = 'Winner Group A'
                data[0].loc[24, 'away'] = 'Runner-up Group B'
                data[0].loc[25, 'home'] = 'Winner Group B'
                data[0].loc[25, 'away'] = 'Runner-up Group A'
                data[0].loc[26, 'home'] = 'Winner Group D'
                data[0].loc[26, 'away'] = 'Runner-up Group C'
                data[0].loc[27, 'home'] = 'Winner Group C'
                data[0].loc[27, 'away'] = 'Runner-up Group D'
                
                data[0].loc[28, 'home'] = 'Winner Match 25'
                data[0].loc[28, 'away'] = 'Winner Match 26'
                data[0].loc[29, 'home'] = 'Winner Match 27'
                data[0].loc[29, 'away'] = 'Winner Match 28'

                data[0].loc[30, 'home'] = 'Loser Match 29'
                data[0].loc[30, 'away'] = 'Loser Match 30'

                data[0].loc[31, 'home'] = 'Winner Match 29'
                data[0].loc[31, 'away'] = 'Winner Match 30'

            return data[0] #Retornamos la data del fixture limpia
        
        elif mode == 'data': #Limpieza de los datos históricos
            #Unimos los datos recolectados con ambas herramientas de WebScraping
            df_complete_data = pd.concat([data[1], data[2]], ignore_index=True)
            df_complete_data.drop_duplicates(inplace=True) #Eliminamos todos los posibles duplicados
            df_complete_data.sort_values('year', inplace=True) #Organizamos los datos en orden ascendente

            df_complete_data['score'] = df_complete_data['score'].str.strip() #Borramos espacios en blanco
            df_complete_data['score'] = df_complete_data['score'].str.replace('[^\d–]', '', regex=True)
            
            #Borramos espacios en blanco de los valores de local y visitante
            df_complete_data['home'] = df_complete_data.home.str.strip()
            df_complete_data['away'] = df_complete_data.away.str.strip()

            df_complete_data[['home_goals', 'away_goals']] = df_complete_data['score'].str.split('–', expand=True)
            df_complete_data.drop('score', axis=1, inplace=True)

            #Asignamos los valores de los goles como tipo entero
            df_complete_data = df_complete_data.astype({'home_goals': int, 'away_goals':int, 'year':int})

            #Separamos en dos DataFrame para el local y el visitante respectivamente
            df_home =  df_complete_data[['home', 'home_goals', 'away_goals']]
            df_away =  df_complete_data[['away', 'home_goals', 'away_goals']]

            #Renombramos las columnas para cada DataFrame creado
            df_home = df_home.rename(columns={'home':'Team', 'home_goals':'Goals_Scored', 'away_goals':'Goals_Conceded'})
            df_away = df_away.rename(columns={'away':'Team', 'home_goals':'Goals_Conceded', 'away_goals':'Goals_Scored'}) 

            #Retornamos la data limpia de los partidos históricos, y también separado en local/visitante
            return df_complete_data, df_home, df_away
        
    def teamStrength(self, df_h, df_a):
        #La fuerza del equipo está relacionada con el promedio histórico de goles anotados y recibidos
        df_team_strength = pd.concat([df_h, df_a], ignore_index=True).groupby('Team').mean()
        return df_team_strength

In [16]:
#Creamos el objeto que nos permite limpiar la data
Tablas = DataCleaner('Programacion_Copa_America_2024.csv',
                    'Conmebol_Copa_America_initial_data.csv',
                    'Conmebol_Copa_America_missing_data.csv')

data = Tablas.readData() #Leemos la data
df_fixture = Tablas.cleanData(data) #Obtenemos los datos limpios del Fixture de la Copa América 2024
df_data = Tablas.cleanData(data, 'data') #Obtenemos los datos limpios de los partidos históricos de la copa
df_complete_data = df_data[0] #Los datos históricos completos
#Los datos históricos limpios, separados en local y visitante
df_home = df_data[1]
df_away = df_data[2]
df_strength = Tablas.teamStrength(df_home, df_away)

## **MODELING**

En esta parte del notebook nos encargamos de desarrollar el modelo que simulará el torneo y predecirá el campeón de la competencia. Usamos la clase Modeling, la cual se encarga de desarrollar todos los pasos necesarios para la simulación del torneo y la predicción del ganador.

In [29]:
class Modeling:
    '''
    Clase encargada de realizar la simulación del torneo, desde la fase de grupos hasta las fases eliminatorias
    (Cuartos de final, Semifinal) y la final. Basando la simulación en una conocida función densidad de
    probabilidad.

    Atributos:
        - tablas: Conjunto de DataFrames con las tablas de posiciones de los grupos de la Copa.

        - fuerza: DataFrame con la información de la fuerza de los equipos.

    Métodos:
        - teamStrength: Se encarga de presentar la fuerza de cada equipo de la Copa.
        
        - Rounds: Se encarga de separar el fixture en las diferentes fases del torneo.

        - predictPoints: Se encarga de realizar la predicción de los puntos de los equipos.

        - getWinner: Se encarga de asignar al ganador de los partidos en cada fase. 
        
        - uptadeTable: Se encarga de actualizar las tablas, una vez actualizadas las rondas

        - simulateTournament: Se encarga de simular la fase de grupos, y simular las diferentes fases del torneo
        
    '''
    
    def __init__(self, tabla, df_s):
        self.tablas = tabla
        self.fuerza = df_s
    
    def Rounds(self, df_f):
        #Separamos el fixture en las diferentes fases del torneo
        df_groups = df_f[:24].copy() #Fase de grupos
        df_fixture_quarter = df_f[24:28].copy() #Cuartos de Final 
        df_fixture_semi = df_f[28:30].copy() #Semifinal 
        df_fixture_final = df_f[31:].copy() #Final

        return [df_groups, df_fixture_quarter, df_fixture_semi, df_fixture_final]

    def predictPoints(self, home, away):
        if home in self.fuerza.index and away in self.fuerza.index:
            #goals_scored*goals_conceded
            #Creamos los parámetros lambda para la distribución de probabilidad de home/away
            lamb_home = self.fuerza.at[home, 'Goals_Scored'] * self.fuerza.at[away, 'Goals_Conceded']
            lamb_away = self.fuerza.at[away, 'Goals_Scored'] * self.fuerza.at[home, 'Goals_Conceded']
            #Inicializamos la probabilidad de victoria, empate y derrota en cero
            prob_home, prob_away, prob_draw = 0, 0, 0

            for i in range(7): #Goles equipo local
                for j in range(7): #Goles equipo visitante
                    #Obtenemos la probabilidad de que el equipo local marque i goles y el visitante j goles
                    p = poisson.pmf(i, lamb_home) * poisson.pmf(j, lamb_away)
                    
                    #En casos de empate 
                    if i == j:
                        prob_draw += p
                    #En casos de que gane el local
                    elif i > j:
                        prob_home += p
                    #En casos de que gane el visitante
                    else:
                        prob_away += p
        
            #Retornamos la predicción de puntos para el local y el visitante
            points_home = 3 * prob_home + prob_draw
            points_away = 3 * prob_away + prob_draw

            return (points_home, points_away)
        else:
            return (0,0)
        
    def getWinner(self, df_updated):
        for index, row in df_updated.iterrows():
            home, away = row['home'], row['away']
            points_home, points_away = self.predictPoints(home, away)
            if points_home > points_away:
                winner = home
            else:
                winner = away
                
            df_updated.loc[index, 'Winner'] = winner
        return df_updated
    
    def uptadeTable(self, df_fixture_1, df_fixture_2):
        for index, row in df_fixture_1.iterrows():
            winner = df_fixture_1.loc[index, 'Winner']
            match = df_fixture_1.loc[index, 'score']
            df_fixture_2.replace({f'Winner {match}':winner}, inplace=True)
            
        df_fixture_2['Winner'] = ''
        return df_fixture_2
    
        
    def simulateTournament(self, df):#[df_groups, df[1], df[2] df[3]]):
        #Creamos un ciclo for que recorra cada grupo de la Copa América en las tablas
        for group in self.tablas:
            teams_in_group = self.tablas[group]['Team'].values #Extraemos los equipos de cada grupo
            #Filtramos los 6 partidos jugados por cada grupo 
            df_fixture_real_matches = df[0][df[0]['home'].isin(teams_in_group)] 
                
            #Creamos un ciclo for que se itera en cada fila de los 6 partidos de cada grupo 
            for index, row in df_fixture_real_matches.iterrows():
                home, away = row['home'], row['away'] #Extraemos el equipo local y visitante
                #Asignamos los puntos a cada equipo en todos los grupos de la copa
                points_home, points_away = self.predictPoints(home, away)
                #Actualizamos las tablas de cada grupo, con los puntos obtenidos por equipo
                self.tablas[group].loc[self.tablas[group]['Team'] == home, 'Pts'] += points_home
                self.tablas[group].loc[self.tablas[group]['Team'] == away, 'Pts'] += points_away 
        
            self.tablas[group] = self.tablas[group].sort_values('Pts', ascending=False).reset_index() 
            self.tablas[group] = self.tablas[group][['Team', 'Pts']]
            self.tablas[group] = self.tablas[group].round(0)

            group_winner = self.tablas[group].loc[0, 'Team']
            group_runner_up = self.tablas[group].loc[1, 'Team']

            df[1].replace({f'Winner {group}': group_winner,
                           f'Runner-up {group}': group_runner_up}, 
                           inplace=True)
        
        df[1]['Winner'] = ''

        df[1] = self.getWinner(df[1])
    

        df[2]= self.uptadeTable(df[1], df[2])
        df[2] = self.getWinner(df[2])

        df[3] = self.uptadeTable(df[2], df[3])
        df[3] = self.getWinner(df[3])

        df_complete = pd.concat([df[1], df[2], df[3]], ignore_index=True)

        return df_complete

In [30]:
Simulator = Modeling(tabla_ca, df_strength)
tournament = Simulator.Rounds(df_fixture)
final = Simulator.simulateTournament(tournament)

In [31]:
final['Fase'] = ''
for i in range(4):
    final.loc[i, 'Fase'] = 'Cuartos de Final'
final.loc[4, 'Fase'] = 'Semi-Final'
final.loc[5, 'Fase'] = 'Semi-Final'
final.loc[6, 'Fase'] = 'Final'

In [32]:
final

,home,score,away,year,Winner,Fase
0,Argentina,Match 25,Ecuador,2024,Argentina,Cuartos de Final
1,Mexico,Match 26,Peru,2024,Mexico,Cuartos de Final
2,Brazil,Match 27,United States,2024,Brazil,Cuartos de Final
3,Uruguay,Match 28,Paraguay,2024,Uruguay,Cuartos de Final
4,Argentina,Match 29,Mexico,2024,Argentina,Semi-Final
5,Brazil,Match 30,Uruguay,2024,Brazil,Semi-Final
6,Argentina,Match 32,Brazil,2024,Argentina,Final
